# PA 6 - Decision Trees

## Step 1 - Decision Tree Classifier

This Assignment is mainly focused on using decision trees to classify instances. As such, our first step is to create a function to generate these decision trees.

First, we will need to import several modules later in this code, so we will do that here.

In [61]:
import math
import operator
import copy
import random
from tabulate import tabulate

Then, we will need several helper functions that will be used in the `tdidt()` classifier.

The first set are functions that are copied from the functions developed in class. We will pull `group_by()` and `get_column()` from previous repositories, as well as `partition_instances()` from the starter code for this assignment located in the `DecisionTreeFun` repository.

In [62]:
# @Gina's Repo
def groupBy(table, column_index, include_only_column_index=None):
    # first identify unique values in the column
    group_names = sorted(list(set(get_column(table, column_index))))

    # now, we need a list of subtables
    # each subtable corresponds to a value in group_names
    # parallel arrays
    groups = [[] for name in group_names]
    for row in table:
        # which group does it belong to?
        group_by_value = row[column_index]
        index = group_names.index(group_by_value)
        if include_only_column_index is None:
            groups[index].append(row.copy()) # note: shallow copy
        else:
            groups[index].append(row[include_only_column_index])

    return group_names, groups

# takes a table and a column index
# returns a column at index where values are converted to numeric
def get_column(table, column_index):
    column = []
    for item in table:
        column.append(item[column_index])
    return column

def partition_instances(instances, att_index, att_domain):
    # this is a group by att_domain, not by att_values in instances
    partition = {}
    for att_value in att_domain:
        subinstances = []
        for instance in instances:
            # check if this instance has att_value at att_index
            if instance[att_index] == att_value:
                subinstances.append(instance)
        partition[att_value] = subinstances
    return partition

We will also need to develop several helper functions of our own to use. These are defined below:

* `check_all_same_att()`
    * **Parameters**:
        * `instances` - A list of the current partitioned instances to check
        * `index` - The index to query
    * **Returns**:
        * `True` if all instances in `instances` have the same value at `index`; `False` if else.
* `check_all_same_class()`
     * **Parameters**:
        * `instances` - A list of the current partitioned instances to check
        * `class_index` - The index of the classifying variable
    * **Returns**:
        * `True` if all instances in `instances` have the same class; `False` if else.

* `select_attribute()`
    * **Parameters**:
        * `instances` - A list of the current partitioned instances
        * `att_indexes` - A list of valid indices to split on
        * `class_index` - The index of the classifying attribute
    * **Returns**:
        * Uses the calculations for Entropy and Information Gain discussed in class to return the index attribute with the lowest entropy to be split on next
* `handle_clash()`
    * **Parameters**:
        * `instances` - A list of the current partitioned instances
        * `class_index` - The index of the classifying attribute
    * **Returns**:
        * Uses majority voting to resolve clashes in the decision tree and create a Leaf Node with the most frequent class value

In [63]:
def check_all_same_att(instances, index):
    base = instances[0][index]
    for elem in instances:
        if elem[index] != base:
            return False
    return True

def check_all_same_class(instances, class_index):
    base = instances[0][class_index]
    for elem in instances:
        if elem[class_index] != base:
            return False
    return True

def select_attribute(instances, att_indexes, class_index):
    Entropy_list = {}
    for index in att_indexes:
        E_new = 0
        names, values = groupBy(instances, index)
        for val in values:
            ratios = {}
            total = 0
            for instance in val:
                if instance[class_index] not in ratios:
                    ratios[instance[class_index]] = 1
                else:
                    ratios[instance[class_index]] += 1
                total += 1
            E = 0
            for ratio in ratios:
                E += (ratios[ratio] / total) * math.log((ratios[ratio] / total), 2)
            E_new += (total / len(instances)) * -E
        Entropy_list[index] = E_new

    min_i = att_indexes[0]
    for index in att_indexes:
        if Entropy_list[index] < Entropy_list[min_i]:
            min_i = index
    return min_i

def handle_clash(instances, class_index):
    votes = {}
    for instance in instances:
        if instance[class_index] not in votes:
            votes[instance[class_index]] = 1
        else:
            votes[instance[class_index]] += 1
    # Referenced from https://stackoverflow.com/questions/613183/how-do-i-sort-a-dictionary-by-value
    sorted_x = sorted(votes.items(), reverse=True, key=operator.itemgetter(1))
    return ["Leaf", sorted_x[0][0]]


Now that we have these classifiers, we can construct our `tdidt()` decision tree generator, as well as the `classify_tdidt()` classifier.

* `tdidt()`
    * **Parameters**:
        * `instances` - The currently partitioned instances. On first recursive call, these are initialized as the entire dataset.
        * `att_indexes` - A list of all valid indices to split on. On first recursive call, these are initialized as all attribute indices.
        * `att_domains` - A list of all valid values for each attribute in the dataset.
        * `class_index` - The index of the classifying attribute.
    * **Returns**:
        * A Decision Tree, represented using nested lists
* `classify_tdidt()`
    * **Parameters**:
        * `tree` - A decision tree, generated by `tdidt()`
        * `instance` - The unseen instance to classify
    * **Returns**:
        * A predicted classification using the decision tree

In [64]:
def tdidt(instances, att_indexes, att_domains, class_index):
    if check_all_same_class(instances, class_index):
        return ["Leaf", instances[0][class_index]]
    if att_indexes == []:
        return handle_clash(instances, class_index)
    index = select_attribute(instances, att_indexes, class_index)
    new_indexes = att_indexes[:]
    new_indexes.remove(index)
    if check_all_same_att(instances, index):
        return tdidt(instances, new_indexes, att_domains, class_index)
    else:
        tree = ["Attribute", index]
        partitions = partition_instances(instances, index, att_domains[index])
        for val in partitions:
            if (partitions[val] == []):
                return handle_clash(instances, class_index)
            tree.append(["Value", val, tdidt(partitions[val], new_indexes, att_domains, class_index)])
        return tree

def classify_tdidt(tree, instance):
    if tree[0] == 'Leaf':
        return tree[1]
    else:
        i = 2
        while (instance[tree[1]] != tree[i][1]):
            i += 1
        return classify_tdidt(tree[i][2], instance)


To test our classifier, we will make use of the "interview" dataset provided in class.

In [65]:
table = [
        ["Senior", "Java", "no", "no", "False"],
        ["Senior", "Java", "no", "yes", "False"],
        ["Mid", "Python", "no", "no", "True"],
        ["Junior", "Python", "no", "no", "True"],
        ["Junior", "R", "yes", "no", "True"],
        ["Junior", "R", "yes", "yes", "False"],
        ["Mid", "R", "yes", "yes", "True"],
        ["Senior", "Python", "no", "no", "False"],
        ["Senior", "R", "yes", "no", "True"],
        ["Junior", "Python", "yes", "no", "True"],
        ["Senior", "Python", "yes", "yes", "True"],
        ["Mid", "Python", "no", "yes", "True"],
        ["Mid", "Java", "yes", "no", "True"],
        ["Junior", "Python", "no", "yes", "False"]
    ]

We will first generate a decision tree using `tdidt()`, and then we will classify two instances: ("Senior", "Java", "no", "yes"), which exists in the dataset and should have the classification ("False"); and ("Junior", "Java", "no", "no"), which is not in the dataset but based on our in-class tree, should have the classification ("True")

In [66]:
tree = tdidt(table, [0, 1, 2, 3], [["Senior", "Mid", "Junior"], ["Java", "Python", "R"], ["no", "yes"], ["no", "yes"]], 4)

print(classify_tdidt(tree, ["Senior", "Java", "no", "yes"]))
print(classify_tdidt(tree, ["Junior", "Java", "no", "no"]))

False
True


## Step 2 - Titanic Data

This step asks us to use our classifier for the titanic dataset, and to test it using stratified 10-fold cross-validation.

To do this, the first step will be to import the dataset and clean it as we have in previous assignments.

In [67]:
def read_data(filename):
    f = open(filename, 'r')
    text = f.read()
    f.close()
    return text

def numerify_instance(instance):
    new = []
    for attribute in instance:
        try:
            new.append(float(attribute))
        except:
            new.append(attribute)
    return new

def create_dataset(data):
    data_r = data.splitlines()
    dataset_r = []
    for line in data_r:
        instance = line.split(',')
        dataset_r.append(instance)
    dataset = []
    for instance in dataset_r:
        newInstance = numerify_instance(instance)
        dataset.append(newInstance)
    return dataset

def resolve_missing_values(data):
    for i in range(10):
        if i != 8:
            sum_i = 0
            count_i = 0
            for instance in data:
                if instance[i] != "NA":
                    try:
                        sum_i += instance[i]
                        count_i += 1
                    except:
                        print(instance[i])
            if count_i == 0:
                continue
            mean = sum_i / count_i
            for instance in data:
                if instance[i] == "NA":
                    instance[i] = mean
                    
titanic_data = create_dataset(read_data("titanic_data.txt"))

We will also reuse the `create_cross_fold()` function from PA4 and PA5 for creating subsamples.

In [68]:
def create_cross_fold(data, n):
    data_r = copy.deepcopy(data)
    random.shuffle(data_r)
    size = int(len(data) * 1/n) 
    start = 0
    end = size
    folds = []
    for i in range(n-1):
        folds.append(data[start:end])
        start = end + 1
        end += size + 1
    folds.append(data[start:])
    return folds

Now we will do our cross-fold testing. We will write a function, `confusion_matrix_tdidt()` to both do the classification and generate a confusion matrix.

In [102]:
def confusion_matrix_titanic(data, att_indexes, att_domains, class_index, n):
    results = [["no", 0, 0], ["yes", 0, 0]]
    folds = create_cross_fold(data, n)
    for i in range(n):
        test = folds[i]
        train = []
        for x in range(n):
            if x == i:
                continue
            train += folds[x]
        curr_tree = tdidt(train, att_indexes, att_domains, class_index)
        predictions = [classify_tdidt(curr_tree, instance[:3]) for instance in test]
        actual = [instance[3] for instance in test]
        for i in range(len(actual)):
            if actual[i] == "no":
                if predictions[i] == "no":
                    results[0][1] += 1
                else:
                    results[0][2] += 1
            else:
                if predictions[i] == "no":
                    results[1][1] += 1
                else:
                    results[1][2] += 1
    for row in results:
        row.append(sum(row[1:]))
        if row[0] == "no":
            row.append((row[1] / row[3]) * 100)
        else:
            row.append((row[2] / row[3]) * 100)
    header = ["Survived", "no", "yes", "Total", "Recognition (%)"]
    print("Decision Tree (Stratefied ", n, "-Fold Cross Validation Results):", sep="")
    print("===============================================================")
    print(tabulate(results, headers=header))
    

Finally, we can do our classification and generate a confusion matrix.

In [103]:
confusion_matrix_titanic(titanic_data, [0, 1, 2], [["crew", "third", "second", "first"], ["adult", "child"], ["male", "female"], ["yes", "no"]], 3, 10)

Decision Tree (Stratefied 10-Fold Cross Validation Results):
Survived      no    yes    Total    Recognition (%)
----------  ----  -----  -------  -----------------
no           268    440      708            37.8531
yes           20   1464     1484            98.6523


It is interesting to see that this classifier is amazing at predicting people that survived, but is terrible at predicting people that did not survive. Overall, this classifier is comparable to the one in PA5, which has a slightly better No prediction rate, but a slightly worse Yes rate.

## Step 2 - MPG Classification

For this step, we will use our `tdidt()` decision tree to classify mpg data from the auto dataset. As before, our first step is to import and clean the dataset.

In [71]:
auto_data_r = create_dataset(read_data("auto-data.txt"))
resolve_missing_values(auto_data_r)

Then, we will restrict what attributes are included in our dataset. For the purpose of this assignment, we are concerned with the cylinders, weight, and model year attributes, as well as mpg as a classifier. We will copy `clean_auto_data()` from PA5 to accomplish this.

In [72]:
def clean_auto_data(data):
    cleaned_auto_data = []
    for instance in data:
        cleaned_instance = [instance[1], instance[4], instance[6], instance[0]]
        cleaned_auto_data.append(cleaned_instance)
    return cleaned_auto_data

auto_data_c = clean_auto_data(auto_data_r)

Finally, as we did in PA5, we need to discretize the continuous attributes weight and cylinder. We will do this using the NHSTA vehicle size classification table and the DOE mpg classification rating, provided below.

| Rating | MPG   |
|--------|-----  |
|   10   | ≥ 45  |
|   9    | 37-44 |
|   8    | 31-36 |
|   7    | 27-30 |
|   6    | 24-26 |
|   5    | 20-23 |
|   4    | 17-19 |
|   3    | 15-16 |
|   2    |   14  |
|   1    | ≤ 13  |

| Ranking |  Weight   |
|---------|-----------|
|    5    | ≥ 3500    |
|    4    | 3000-3499 |
|    3    | 2500-2999 |
|    2    | 2000-2499 |
|    1    | ≤ 1999    |

In [73]:
def mpg_to_DOE(mpg):
    if mpg >= 45:
        y = 10
    elif mpg >= 37:
        y = 9
    elif mpg >= 31:
        y = 8
    elif mpg >= 27:
        y = 7
    elif mpg >= 24:
        y = 6
    elif mpg >= 20:
        y = 5
    elif mpg >= 17:
        y = 4
    elif mpg >= 15:
        y = 3
    elif mpg >= 14:
        y = 2
    else:
        y = 1
    return y

def weight_to_NHTSA(weight):
    if weight >= 3500:
        return 5
    elif weight >= 3000:
        return 4
    elif weight >= 2500:
        return 3
    elif weight >= 2000:
        return 2
    else:
        return 1

    
def discretize_auto_data(data):
    discrete_data = []
    for instance in data:
        discrete = [instance[0], weight_to_NHTSA(instance[1]), instance[2], mpg_to_DOE(instance[3])]
        discrete_data.append(discrete)
    return discrete_data

auto_data = discretize_auto_data(auto_data_c)

Now, we will rework our confusion matrix generator to format for the auto dataset.

In [74]:
def confusion_matrix_auto(data, att_indexes, att_domains, class_index, n):
    results = [["1", 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
               ["2", 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
               ["3", 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
               ["4", 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
               ["5", 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
               ["6", 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
               ["7", 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
               ["8", 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
               ["9", 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
               ["10", 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

    folds = create_cross_fold(data, n)
    for i in range(n):
        test = folds[i]
        train = []
        for x in range(n):
            if x == i:
                continue
            train += folds[x]
        curr_tree = tdidt(train, att_indexes, att_domains, class_index)
        predictions = [classify_tdidt(curr_tree, instance[:3]) for instance in test]
        actual = [instance[3] for instance in test]
        for i in range(len(actual)):
            y = actual[i]
            x = predictions[x]
            results[y-1][x] += 1
    i = 1
    for row in results:
        row.append(sum(row[1:]))
        if row[11] == 0:
            row.append("NaN")
        else:
            row.append((row[i] / row[11]) * 100)
        i += 1
    header = ["MPG Rating", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "Total", "Recognition (%)"]
    print("Decision Tree (Stratefied ", n, "-Fold Cross Validation Results):", sep="")
    print("===============================================================")
    print(tabulate(results, headers=header))


In [75]:
confusion_matrix_auto(auto_data, [0, 1, 2], [[4.0, 6.0, 8.0], [1, 2, 3, 4, 5], [70, 71, 72, 73, 74, 75, 76, 77, 78, 79], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]], 3, 10)

Decision Tree (Stratefied 10-Fold Cross Validation Results):
  MPG Rating    1    2    3    4    5    6    7    8    9    10    Total    Recognition (%)
------------  ---  ---  ---  ---  ---  ---  ---  ---  ---  ----  -------  -----------------
           1   14    1    0    1    6    3    4    3    0     0       32           43.75
           2   13    0    0    1    2    0    1    1    2     0       20            0
           3   13    0    1   13    1    5    3    0    2     0       38            2.63158
           4   15    2    0   16    3    6    9    2    1     0       54           29.6296
           5   16    0    1   21    1    6   10    5    3     0       63            1.5873
           6   15    3    1    8    3    3    5    3    0     0       41            7.31707
           7    5    1    3    8    0    7    5    0    4     0       33           15.1515
           8    1    0    3    8    1    1    2    3    3     0       22           13.6364
           9    0    0    0    2

As we can see, this decision tree is all over the board with its classifications. Some, like predicting MPG class 1, are fairly good, while others like class 2 and 9 are absolutely awful. This makes it tough to compare directly to the classifier in PA5, but overall I would say that it is slightly worse, as only one classification gets above a 40% prediction rate, with the average being closer to 15-20%.

## Step 4 - Rule Generation

For this step, we will attempt to visualize the rules that our decision tree is actually using to classify instances. To do this, we will need to define a rule inference funtion, which we will call `print_rules()`. This will be a recursive function which will loop through the nested list structure of the tree we generate and will print out the underlying rules that define it.

In [98]:
def print_rules(tree, attributes, string):
    if tree[0] == "Attribute":
        if string == "":
            string += "IF " + str(attributes[tree[1]]) + " == "
        else:
            string += " AND " + str(attributes[tree[1]]) + " == "
        for value in tree[2:]:
            valString = string + str(value[1])
            print_rules(value[2], attributes, valString)
    elif tree[0] == "Leaf":
        string += " THEN class == " + str(tree[1])
        print(string)

Now we will create overall data trees for the titanic and auto datasets, and see what rules the trees generate.

First, we will generate a tree for the titanic dataset:

In [99]:
titanic_tree = tdidt(titanic_data, [0, 1, 2], [["crew", "third", "second", "first"], ["adult", "child"], ["male", "female"], ["yes", "no"]], 3)
print_rules(titanic_tree, ["Class", "Age", "Sex", "Survived"], "")

IF Sex == male AND Class == crew AND Age == adult THEN class == no
IF Sex == male AND Class == crew AND Age == child THEN class == no
IF Sex == male AND Class == third THEN class == no
IF Sex == male AND Class == second AND Age == adult THEN class == yes
IF Sex == male AND Class == second AND Age == child THEN class == yes
IF Sex == male AND Class == first AND Age == adult THEN class == no
IF Sex == male AND Class == first AND Age == child THEN class == no
IF Sex == female AND Class == crew AND Age == adult THEN class == yes
IF Sex == female AND Class == crew AND Age == child THEN class == no
IF Sex == female AND Class == third THEN class == yes
IF Sex == female AND Class == second AND Age == adult THEN class == yes
IF Sex == female AND Class == second AND Age == child THEN class == yes
IF Sex == female AND Class == first AND Age == adult THEN class == yes
IF Sex == female AND Class == first AND Age == child THEN class == no


And now for the auto dataset:

In [100]:
auto_tree = tdidt(auto_data, [0, 1, 2], [[4.0, 6.0, 8.0], [1, 2, 3, 4, 5], [70, 71, 72, 73, 74, 75, 76, 77, 78, 79], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]], 3)
print_rules(auto_tree, ["Cylinders", "Weight Class", "Model Year", "MPG"], "")

IF Weight Class == 1 THEN class == 8
IF Weight Class == 2 AND Model Year == 70 THEN class == 6
IF Weight Class == 2 AND Model Year == 71 THEN class == 7
IF Weight Class == 2 AND Model Year == 72 THEN class == 5
IF Weight Class == 2 AND Model Year == 73 THEN class == 5
IF Weight Class == 2 AND Model Year == 74 THEN class == 6
IF Weight Class == 2 AND Model Year == 75 THEN class == 7
IF Weight Class == 2 AND Model Year == 76 THEN class == 7
IF Weight Class == 2 AND Model Year == 77 THEN class == 7
IF Weight Class == 2 AND Model Year == 78 THEN class == 7
IF Weight Class == 2 AND Model Year == 79 THEN class == 8
IF Weight Class == 3 AND Model Year == 70 THEN class == 5
IF Weight Class == 3 AND Model Year == 71 THEN class == 4
IF Weight Class == 3 AND Model Year == 72 THEN class == 5
IF Weight Class == 3 AND Model Year == 73 THEN class == 4
IF Weight Class == 3 AND Model Year == 74 THEN class == 4
IF Weight Class == 3 AND Model Year == 75 THEN class == 5
IF Weight Class == 3 AND Model Year

It is interesting to note that, while the titanic ruleset had varied and interesting rule splits, the auto dataset strictly splits on Weight Class and then Model Year for every single rule, save for the two that jump straight to a prediction after the first split.